In [7]:
import pandas as pd
import altair as alt
import numpy as np

In [8]:
import os
files = os.listdir()
files = [x for x in files if 'API' in x]
#ww2test = pd.read_excel('statistic_id1261433_gender-ratios-in-select-countries-after-the-second-world-war-1950.xlsx',sheet_name='Data',skiprows=4)



combo = pd.read_csv(files[0],skiprows=4)
combo = combo.melt(id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], var_name="Date", value_name="Value")
combo = combo.loc[:,['Country Name','Country Code','Date','Value']].dropna()
combo.columns = combo.columns.str.replace('Value', files[0][:20])

for i in range(1,len(files)):
    world = pd.read_csv(files[i],skiprows=4)
    world = world.melt(id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], var_name="Date", value_name="Value")
    world = world.loc[:,['Country Name','Country Code','Date','Value']].dropna()
    
    combo = pd.merge(combo, world[['Country Code','Date', 'Value']], on=['Country Code','Date'])
    combo.columns = combo.columns.str.replace('Value', files[i][:20])

combo

,Country Name,Country Code,Date,API_SP.POP.TOTL_DS2_,API_NY.GDP.MKTP.PP.C
0,Aruba,ABW,1990,65712.0,1.363677e+09
1,Africa Eastern and Southern,AFE,1990,309890664.0,5.637251e+11
2,Africa Western and Central,AFW,1990,206739024.0,3.616426e+11
3,Angola,AGO,1990,11828638.0,3.883544e+10
4,Albania,ALB,1990,3286542.0,8.378167e+09
...,...,...,...,...,...
7725,Samoa,WSM,2022,222382.0,1.354208e+09
7726,Kosovo,XKX,2022,1761985.0,2.637895e+10
7727,South Africa,ZAF,2022,59893885.0,9.535361e+11
7728,Zambia,ZMB,2022,20017675.0,7.958228e+10


In [9]:
#alt.data_transformers.disable_max_rows()
combo['GDP'] = combo['API_NY.GDP.MKTP.PP.C'] / 1e10
combo['pop'] = combo['API_SP.POP.TOTL_DS2_'] / 1e10

brush = alt.selection_interval(
    encodings=['x'] # limit selection to x-axis (year) values
)

# dynamic query histogram
years = alt.Chart(combo).mark_bar().add_selection(
    brush
).encode(
    alt.X('Date:T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)



#scatter plot, modify opacity based on selection
ratings = alt.Chart(combo).mark_circle().encode(
    #x='API_SP.POP.TOTL_DS2_',
    #y='API_NY.GDP.MKTP.PP.C',
    x='GDP',
    y='pop',
    tooltip='Country Name:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

#ratings
alt.vconcat(years, ratings).properties(spacing=5)

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

See https://altair-viz.github.io/user_guide/large_datasets.html for information on how to plot large datasets, including how to install third-party data management tools and, in the right circumstance, disable the restriction

alt.VConcatChart(...)